# Prepare

In [2]:
! pip install pyexcel pyexcel-xlsx

     |████████████████████████████████| 88 kB 6.5 MB/s 
     |████████████████████████████████| 44 kB 2.2 MB/s 
     |████████████████████████████████| 242 kB 57.8 MB/s 
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 2.5.9
    Uninstalling openpyxl-2.5.9:
      Successfully uninstalled openpyxl-2.5.9


In [13]:
# create multiple xlsx files

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.28
"""

# import
import time
import random
import os
import pyexcel as px
import shutil

#
print("Process Start.")
start_time = time.time()

#
NUM_SAMPLES = 1000

#
alphabet_samples = "abcdefghizklmnopqrstuvwxyz1234567890"

first_name_samples = "김이박최정강조윤장임"
middle_name_samples = "민서예지도하주윤채현지"
last_name_samples = "준윤우원호후서연아은진"

#
def random_string(length):
    result = ""

    for i in range(length):
        result += random.choice(alphabet_samples)

    return result


def random_name():
    result = ""

    result += random.choice(first_name_samples)
    result += random.choice(middle_name_samples)
    result += random.choice(last_name_samples)

    return result

#
if os.path.exists("personal_info"):
    shutil.rmtree("personal_info")
os.mkdir("personal_info")

#
HEADER = ["name", "age", "e-mail", "division", "telephone", "sex"]

#
for i in range(NUM_SAMPLES):

    #
    name = random_name()

    #
    filename = "personal_info/" + str(i) + "_" + name + ".xlsx"

    #
    contents = []

    contents.append(name)
    contents.append(str(time.time())[-2:])
    contents.append(random_string(8) + "@bhban.com")
    contents.append(random_string(3))
    contents.append("010-" + str(time.time())[-4:] + "-" + str(time.time())[-6:-2])
    contents.append(random.choice(["male", "female"]))

    #
    RESULT = [HEADER, contents]

    #
    px.save_as(array=RESULT, dest_file_name=filename)

#
print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code 7.814866542816162 seconds.


# RPA design code
- 목표 정하기
  1. 몇 퍼센트의 파일을 파괴할 것인지 입력받기
  1. 무작위 열을 선택하여 삭제하기
  1. 무작위 열을 선택하여 복제하기
  1. 무작위 열을 선택하여 내용물 조작하기
- 목표를 달성하는 데 필요한 작업 쪼개기
  1. 코드를 실행할 때 사용자에게 퍼센트 입력 받기
  1. 입력 받은 만큼의 파일만을 파괴하기
  1. 엑셀 파일을 읽어와 하나의 열을 무작위로 선택
  1. 엑셀 파일의 열을 삭제
  1. 엑셀 파일의 열을 추가
  1. 엑셀 파일의 열을 선택하여 내용물을 조작

#### 1. 코드를 실행할 때 사용자에게 퍼센트 입력 받기
- ```sys.argv```: 코드를 실행하는 과정에서 사용자에게 추가적인 명령을 입력받는 방법
- 단, ```argv```로 입력 받은 데이터는 문자 형태임에 주의

#### 2. 입력 받은 만큼의 파일만을 파괴하기
1. 파일 목록을 불러온 다음, 순서를 섞는다.
1. 전체 파일 수에 퍼센트를 곱해 몇 개의 파일을 파괴할지 계산한다.
1. 슬라이싱을 통해 파괴할 개수만큼의 파일만 불러와 작업을 수행한다.

#### 3. 엑셀 파일을 읽어와 하나의 열을 무작위로 선택
- ```random.randint(0, 5)```
- 파일에 몇 개의 열이 있을까?
```py
num_columns = len(data_array[0])
random.randint(0, num_columns - 1)
```

#### 4. 엑셀 파일의 열을 삭제
- ```file.delete_columns([1, 3, 5])```

#### 5. 엑셀 파일의 열을 복제 및 추가 (리스트의 복제)
- ```duplicated_column = file.column[0].copy()```
- ```file.column += duplicated_column```

#### 6. 엑셀 파일의 내용물을 모두 같은 값으로 변경하기 (Comprehension) 

In [6]:
a = [1, 2, 3, 4, 5]
CAT = ["고양이" for i in range(len(a))]
CAT

['고양이', '고양이', '고양이', '고양이', '고양이']

In [14]:
# full code

#-*-coding:euc-kr

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.11.28
"""

# import
import time
import os
import pyexcel as px
import sys
import random

#
print("Process Start.")
start_time = time.time()

"""
# original

directory = sys.argv[1]
percent = float(sys.argv[2]) / 100
"""

# revised
directory = "personal_info"
percent = 30 / 100

#
input_files = os.listdir(directory)

DESTROY_COUNT = int(len(input_files) * percent)

#
random.shuffle(input_files)

#
destroy_them = input_files[:DESTROY_COUNT]

#
shift_1 = DESTROY_COUNT/3*2
shift_2 = DESTROY_COUNT/3

#
for filename in destroy_them:

    # exception
    if ".xlsx" not in filename:
        continue

    #
    file = px.get_sheet(file_name=directory + "/" + filename)

    os.remove(directory + "/" + filename)

    #
    data_array = file.array

    num_columns = len(data_array[0])
    victim = random.randint(0, num_columns - 1)

    # 1st Disaster
    if DESTROY_COUNT > shift_1:
        file.delete_columns([victim])

    # 2nd Disater
    elif DESTROY_COUNT > shift_2:
        file.column += file.column[victim].copy()
    
    # 3rd Disaster
    else:
        CAT = ["Peter's Kitten" for i in range(file.number_of_rows())]
        file.column[victim] = CAT

    #
    px.save_as(array=data_array, dest_file_name=directory + "/" + filename)

    #
    DESTROY_COUNT -= 1

#
print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Process Start.
Process Done.
Peter's code 4.5108561515808105 seconds.


## Summary
- 조작되지 않은 원본 파일은 ```os.remove(directory + "/" + filename)```로 삭제
- if문이 실행되면 그 뒤에 오는 elif와 else는 모두 생략된다.

# Exercise

##### 결과 확인

In [15]:
destroy_them[:5]

['265_임윤연.xlsx', '41_이하준.xlsx', '723_장채후.xlsx', '506_윤윤원.xlsx', '861_이주윤.xlsx']

In [16]:
import pandas as pd

In [24]:
for filename in destroy_them[:5]:
    df = pd.read_excel(io="personal_info" + "/" + filename)
    print(df)

  name  age              e-mail      telephone   sex
0  임윤연   83  g4gwc32f@bhban.com  010-8421-2984  male
  name  age              e-mail      telephone   sex
0  이하준   72  w96iod1u@bhban.com  010-7089-2170  male
  name  age              e-mail division      telephone
0  장채후   36  nlqzivv9@bhban.com      qch  010-5853-5765
  name  age              e-mail division      telephone
0  윤윤원   45  r0774b9m@bhban.com      o9g  010-2675-0226
  name  age              e-mail division     sex
0  이주윤   24  mm76m232@bhban.com      kkk  female


##### 소숫점이 있는 문자를 숫자로 변환할 때는 주의

In [27]:
float("3.14")

3.14

In [29]:
try:
    int("3.14")
except: ValueError

##### 파일목록 순서 섞기

In [30]:
a = [1, 2, 3, 4]
random.shuffle(a)
a

[4, 2, 3, 1]

##### 무작위 열 선택

In [31]:
num = random.randint(0, 5)
for i in range(3):
    print(num)

2
2
2


In [32]:
for i in range(3):
    print(random.randint(0, 5))

3
2
5
